In [1]:
%run "libraries.ipynb"

from wekeypedia.wikipedia.api import api as api
from __future__ import division
from math import ceil

common libraries loaded


In [2]:
u = wekeypedia.WikipediaPage("Love")
v = wekeypedia.WikipediaPage("Amour", "fr")

convergence = defaultdict(dict)

convergence["en"]["en"] = list(set([ x["title"] for x in u.get_links() ]))
convergence["fr"]["fr"] = list(set([ x["title"] for x in v.get_links() ]))

print len(convergence["en"]["en"])
print len(convergence["fr"]["fr"])

408
473


In [3]:
def api_bunch(page_titles, lang, req):
  results = defaultdict(list)
  param  = req
  
  w = api(lang)

  for i in range(0,int(ceil(len(page_titles)/50))):
    param["titles"] = "|".join(page_titles[i*50:i*50+50-1])
    
    while True:
      r = w.get(param)
      results.update({ p["title"]: p['langlinks'] for pageid, p in r["query"]["pages"].items() if 'langlinks' in p })
      
      if "continue" in r:
        param.update(r["continue"])
      else:
        break

  return results

def get_lang_projection(convergence, source, target):
  params = {
    "redirects": "",
    "format": "json",
    "action": "query",
    "prop": "info|langlinks",
    "lllimit": 500,
    "lllang": target,
    "continue":""
  }
  
  r = api_bunch(convergence[source][source], source, params)

  convergence[source][target] = [ (page, t["*"]) for page,tt in r.items() for t in tt if t["lang"] == target ]
  
  return convergence
  
get_lang_projection(convergence, "en", "fr");
get_lang_projection(convergence, "fr", "en");

In [4]:
print len(convergence["en"]["fr"])
print len(convergence["fr"]["en"])

309
414


In [5]:
def bunch_redirects(pagelist, lang):
  results = set(pagelist)
  
  params = {
    "redirects": "",
    "format": "json",
    "action": "query"
  }
  
  w = api(lang)

  for i in range(0,int(ceil(len(pagelist)/50))):
    # print i*50,i*50+50-1
    params["titles"] = "|".join(pagelist[i*50:i*50+50-1])
    resp = w.get(params)

    if "redirects" in resp["query"]:
      results = results - set([ r["from"] for r in resp["query"]["redirects"]])
      results = results | set([ r["to"] for r in resp["query"]["redirects"]])
  
  return list(results)

convergence["en"]["en"] = bunch_redirects(convergence["en"]["en"], "en")
convergence["fr"]["fr"] = bunch_redirects(convergence["fr"]["fr"], "fr")

In [6]:
def print_h(h):
  display(HTML("<h4>%s</h4>" % h))  

def print_list(l, lang):
  href = u"http://%s.wikipedia.org/wiki/" % lang
  ul = u"<ul>%s</ul>" % "".join([ u"<li><a href=\"{0}{1}\">{1}</li>".format(href, item) for item in l ])
  
  display(HTML(ul))

In [12]:
print_h("english pages that have no french translation")

s = set(convergence["en"]["en"]) - set([ x[0] for x in convergence["en"]["fr"] ])
print_list(s, "en")
print len(s)

87


In [13]:
print_h("french pages that have no english translation")

s = set(convergence["fr"]["fr"]) - set([ x[0] for x in convergence["fr"]["en"] ])
print_list(s, "fr")
print len(s)

51


In [9]:
print_h("english concepts that are on both pages")
print_list(set(convergence["en"]["en"]) & set([ x[1] for x in convergence["fr"]["en"] ]), "en")
print len(set(convergence["en"]["en"]) & set([ x[1] for x in convergence["fr"]["en"] ]))

122


In [10]:
print_h("french concepts that are on both pages")
print_list(set(convergence["fr"]["fr"]) & set([ x[1] for x in convergence["en"]["fr"] ]), "fr")
print len(set(convergence["fr"]["fr"]) & set([ x[1] for x in convergence["en"]["fr"] ]))

125


In [11]:
print_h("pages that have a translation in both languages and are on both pages")
print_list(set([ x[1] for x in convergence["fr"]["en"] ]) & set([ x[0] for x in convergence["en"]["fr"] ]), "en")
print len(set([ x[1] for x in convergence["fr"]["en"] ]) & set([ x[0] for x in convergence["en"]["fr"] ]))

121
